# 938 - Exhausting a color

## Problem Statement

A deck of cards contains $R$ red cards and $B$ black cards.<br>
A card is chosen uniformly randomly from the deck and removed. A second card is then chosen uniformly randomly from the cards remaining and removed.
<ul>
<li>
If both cards are red, they are discarded.</li>
<li>
If both cards are black, they are both put back in the deck.</li>
<li>
If they are different colours, the red card is put back in the deck and the black card is discarded.</li></ul>

Play ends when all the remaining cards in the deck are the same colour and let $P(R,B)$ be the probability that this colour is black.

You are given $P(2,2) = 0.4666666667$, $P(10,9) = 0.4118903397$ and $P(34,25) = 0.3665688069$.

Find $P(24690,12345)$. Give your answer with 10 digits after the decimal point.

## Solution

We can deduce the recurrence relation

\begin{equation}
    P(R, B) = \frac{R(R - 1)}{(R + B)(R + B - 1)}  P(R - 2, B) + 2\frac{BR}{(R + B)(R + B - 1)}  P(R, B - 1) + \frac{B(B - 1)}{(R + B)(R + B - 1)}  P(R, B).
\end{equation}

Rearranging and solving for $P(R, B)$, we get

\begin{equation}
    P(R, B) = \frac{1}{1 - \frac{B(B - 1)}{(R + B)(R + B - 1)}}\left[\frac{R(R - 1)}{(R + B)(R + B - 1)}  P(R - 2, B) + 2\frac{BR}{(R + B)(R + B - 1)}  P(R, B - 1)\right].
\end{equation}

The base cases are 

\begin{align}
    &P(0, 0) = 0, \\
    &P(1, B) = 0, \\
    &P(0, B \neq 0) = 1, \\
    &P(R \neq 0, 0) = 0, \\
    &P(R \neq 0, 1) = 0. \\
\end{align}

We can solve the problem using bottom up dynamic programming. Starting from the base cases, we keep computing the probability for higher $R$ and $B$ values from the lower values. We use numba to speed up the computations. Note that we could optimise space too as we do not need to keep the full table (only two previous rows are needed).

In [1]:

import numpy as np
from numba import njit

@njit
def compute_proba(r, b):
    dp = np.ones((r + 1, b + 1))
    dp[0, 0] = 0
    dp[1:, 0:2] = 0
    dp[1, :] = 0

    for r in range(2, len(dp)):
        for b in range(1, len(dp[0])):
            proba_two_red = (r/(r + b)) * ((r - 1)/(r + b - 1))
            proba_one_red = 2 * (r / (r + b)) * (b / (r + b - 1))
            proba_two_black = ((b / (r + b)) * ((b - 1) / (r + b - 1)))
            dp[r, b] = (proba_two_red * dp[r - 2, b] + proba_one_red * dp[r, b - 1]) * (1 / (1 - proba_two_black))
    return dp[-1][-1]

In [2]:
b = 12345
r = 24690

p = compute_proba(r, b)
round(p, 10)

0.2928967987